# This file runs study 1, no winner case, without spillover effects

In [6]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import repeat
from scipy import stats
import seaborn as sns
import pandas as pd
import scipy.stats as st

In [7]:
def best_coverage(data,n_trials,n_resamples,true_val):
    xb = np.random.choice(data, (n_resamples, n_trials), replace=True)
    low,high=st.t.interval(0.95, len(xb)-1, loc=np.mean(xb), scale=st.sem(xb))
    return np.sum(np.logical_and(true_val >= low, true_val <= high))/n_trials


In [22]:
class MAB:
    def __init__(self, probs, karms, nfirst, nEpisodes):
        '''
        meanings for the parameters:
        probs: True success rate for each arm k in {1, ..., K} 
        kArms: Number of arms to choose among
        nsamples: Number of subjects to test at each time step

        '''
        self.n_obs = 4000
        self.probs = probs
        self.K = karms
        self.n = int((self.n_obs-nfirst)/(nEpisodes-1)) if nEpisodes > 1 else 0
        self.T = nEpisodes
        self.first = nfirst
        self.true_win_arm = np.argmax(np.asarray(probs))

    def best_arm(self, s:np.ndarray, asgn):
        draws = 10000
        # + 1 for a Beta(1, 1) prior
        new_alpha = s+1
        new_beta = asgn-s+1
        selection_table = []
        for i in range(self.K):
            theta = np.random.beta(new_alpha[i], new_beta[i], draws)
            selection_table.append(theta)

        winning_arms = np.argmax(selection_table, axis = 0)
        winning_prob = []
        
#         for i in range(self.K):
#             winning_prob.append(winning_arms.count(i)/draws)
        c = Counter(winning_arms)
        winning_prob = np.zeros(self.K)
        for i in range(self.K):
            winning_prob[i] = c[i]/draws

        return winning_prob


    def reset(self):
        # initialization
        success = np.zeros((self.T, self.K))   # record the success cases of each arm at each time step
        assigned = np.zeros((self.T, self.K)) # record the assigned cases of each arm at each time step
        posterior_probs = 0 # estimated success rate
        winning_probs = np.zeros((self.T, self.K)) # probability to be the true best arm
        # rewards = np.zeros(self.T) # record the average rewards per period
        # regrets = np.zeros(self.T) # record the regrets per period
        return (success, assigned, posterior_probs, winning_probs)


    def do_experiment(self, assignments):
        c = Counter(assignments)
        assigned = [c[i] for i in range(self.K)]
        success = [np.random.binomial(assigned[i], self.probs[i]) for i in range(self.K)]
        return assigned, success


    def Thompson_sampling(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first)
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        # print(adj_probs)
        if self.T > 1:
            count0 = 0
            for i in range(1, self.T):              
                new_assign = np.random.choice(range(self.K), size=self.n, p=list(winning_probs[i-1])) # adaptive design
                new_assigned, new_success = self.do_experiment(new_assign)
                assigned[i] = assigned[i-1]+ new_assigned
                success[i] = success[i-1]+ new_success
                winning_probs[i] = self.best_arm(success[i], assigned[i])
                if winning_probs[i-1,0] == 0:
                    count0 += 1
                else:
                    adj_probs += new_success[0]/winning_probs[i-1,0]/self.n
            posterior_probs = adj_probs / (self.T-count0)
        return (success, assigned, posterior_probs, winning_probs)
    


    def static(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first).tolist()
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        for i in range(1, self.T):
            new_assign = np.random.choice(range(self.K), size=self.n).tolist() # static design
            new_assigned, new_success = self.do_experiment(new_assign)
            assigned[i] = assigned[i-1]+ new_assigned
            success[i] = success[i-1]+ new_success
            winning_probs[i] = self.best_arm(success[i], assigned[i])
            adj_probs += new_success[0]*self.K/self.n
        posterior_probs = adj_probs / self.T


        return (success, assigned, posterior_probs, winning_probs)
        
    def do_replication(self, times, method):
        '''
        parameters:
        final_regrets -- to record the final regret of each replication, in order to compare efficiency
        final_win_arm -- to record the final win arm selected by two methods, in order to compare the accurarcy
        final_win_probs -- to record the final probability of each arm being the best arm, in order to compare the accurarcy
        final_assignment -- to record the final number of assigned subjects to the true best arm, comparing the exploitition
        cum_rewards -- to record the total rewards of every replication
        '''
        # records of the replication
        final_win_arm = np.zeros(times)
        final_win_probs = np.zeros(shape = (times, self.K))
        final_assignment = np.zeros(times) # only record the true best arm
        estimation = np.zeros(times)
        
        if method == 'TS':
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.Thompson_sampling()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs
        else:
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.static()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs

        ate = np.mean(estimation)
        mse = np.mean(np.square(np.subtract(estimation,self.probs[0])))
        rmse = np.sqrt(mse)
        # rmse = np.std(estimation,0)
        low,high=st.t.interval(0.95, len(estimation)-1, loc=np.mean(estimation), scale=st.sem(estimation))
        win_counts = Counter(final_win_arm)
        estimate = {"best_selected":win_counts[0]/times,"ATE":ate, "RMSE":rmse, "Coverage":(low,high)}
        print(estimate)
        return (final_win_arm, final_win_probs, final_assignment, estimation)

In [23]:
# experiment 1
k = 9 # number of treatments
probs = [0.11] + [0.1]*8 # true value
np.random.seed(99332)

In [15]:
# exp 1
first = 4000
periods = 1
sim1 = MAB(probs,k,first, periods)
(s1_ts_final_win_arm, s1_ts_final_win_probs, s1_ts_final_assignment, s1_ts_estimation) = sim1.do_replication(5000, "TS")
(s1_st_final_win_arm, s1_st_final_win_probs, s1_st_final_assignment, s1_st_estimation) = sim1.do_replication(5000, "static")

{'best_selected': 0.2704, 'ATE': 0.10997202856810157, 'RMSE': 0.014742604526863233, 'Coverage': (0.10956325282106438, 0.11038080431513875)}
{'best_selected': 0.27, 'ATE': 0.1099242, 'RMSE': 0.015774115664594322, 'Coverage': (0.10948682727789383, 0.11036157272210617)}


In [24]:
# experiment 3
first = 2000
periods = 2
sim3 = MAB(probs,k,first, periods)
(s3_ts_final_win_arm, s3_ts_final_win_probs, s3_ts_final_assignment, s3_ts_estimation) = sim3.do_replication(5000, "TS")

{'best_selected': 0.2648, 'ATE': 0.10999909014049747, 'RMSE': 0.034576895042637536, 'Coverage': (0.10904035713568867, 0.11095782314530626)}


In [25]:
# experiment 4
first = 800
periods = 5
sim4 = MAB(probs,k,first, periods)
(s4_ts_final_win_arm, s4_ts_final_win_probs, s4_ts_final_assignment, s4_ts_estimation) = sim4.do_replication(5000, "TS")


{'best_selected': 0.2902, 'ATE': 0.11012512594601272, 'RMSE': 0.03743011258541685, 'Coverage': (0.10908728597326409, 0.11116296591876135)}


In [26]:
# experiment 5
first = 400
periods = 10
sim5 = MAB(probs,k,first, periods)
(s5_ts_final_win_arm, s5_ts_final_win_probs, s5_ts_final_assignment, s5_ts_estimation) = sim5.do_replication(5000, "TS")
# (s5_st_final_win_arm, s5_st_final_win_probs, s5_st_final_assignment, s5_st_estimation) = sim5.do_replication(5000, "static")


{'best_selected': 0.2978, 'ATE': 0.11018075587874823, 'RMSE': 0.02721065694482421, 'Coverage': (0.10942628734489988, 0.11093522441259658)}


In [27]:
# experiment 2
first = 200
periods = 20
sim2 = MAB(probs,k,first, periods)
(s2_ts_final_win_arm, s2_ts_final_win_probs, s2_ts_final_assignment, s2_ts_estimation) = sim2.do_replication(5000, "TS")
# (s2_st_final_win_arm, s2_st_final_win_probs, s2_st_final_assignment, s2_st_estimation) = sim2.do_replication(5000, "static")


{'best_selected': 0.2956, 'ATE': 0.10976784083486471, 'RMSE': 0.025671159268538923, 'Coverage': (0.10905607127990068, 0.11047961038982874)}


In [28]:
# experiment 7
first = 80
periods = 50
sim7 = MAB(probs,k,first, periods)
(s7_ts_final_win_arm, s7_ts_final_win_probs, s7_ts_final_assignment, s7_ts_estimation) = sim7.do_replication(5000, "TS")

{'best_selected': 0.3084, 'ATE': 0.10956065049564824, 'RMSE': 0.0236108572125012, 'Coverage': (0.10890609233727286, 0.11021520865402362)}


In [29]:
# exp 6
first = 40
periods = 100
sim6 = MAB(probs,k,first, periods)
(s6_ts_final_win_arm, s6_ts_final_win_probs, s6_ts_final_assignment, s6_ts_estimation) = sim6.do_replication(5000, "TS")

<ipython-input-22-fad731c7549a>:66: RuntimeWarning: invalid value encountered in double_scalars
  posterior_probs = success[0,0]/ assigned[0,0]


{'best_selected': 0.309, 'ATE': 0.11012893049254999, 'RMSE': 0.023237392428755308, 'Coverage': (0.10948462416817575, 0.11077323681692422)}
